# **Capstone - Neighborhoods in Toronto**
## **week 3** Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto<br><br>
### Import Data from URL and to a Pandas-Dataframe

Extract Table with Beautifulsoup and send the table to a DataFrame

In [1]:
!pip install beautifulsoup4

import pandas as pd
from bs4 import BeautifulSoup
import requests

!pip install lxml

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

req=requests.get(url)
soup=BeautifulSoup(req.text,"html.parser")
table = soup.find_all('table')
df=pd.read_html(str(table))[0]

In [3]:
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Ignore cells with a borough that is Not assigned

In [4]:
not_assigned=0

for i, row in df.iterrows():
    if row[2] =='Not assigned':
        print(i,row[0],row[1],row[2])
        not_assigned=1
        
if not_assigned==0:
    print('Everything is fine!')

Everything is fine!


### grouping post codes

In [18]:
df_group = df.groupby('Postal Code')['Neighbourhood'].apply(lambda x:"%s" % ', '.join(x))


df_join= pd.merge(df, df_group, on='Postal Code')
df_join.drop(['Neighbourhood_x'], axis=1, inplace=True)
df_join.drop_duplicates(inplace=True)
df_join.rename(columns={'Neighbourhood_y':'Neighbourhood'}, inplace=True)
df_join.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [19]:
df_join.shape

(103, 3)

### from geocoder.google

In [14]:
!pip install geocoder
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format('Postal Code'))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

KeyboardInterrupt: 

### import geocodes using .CSV file

In [20]:
!wget -O GeoCord.csv http://cocl.us/Geospatial_data/

--2021-01-05 10:00:07--  http://cocl.us/Geospatial_data/
Resolving cocl.us (cocl.us)... 169.63.96.176, 169.63.96.194
Connecting to cocl.us (cocl.us)|169.63.96.176|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/ [following]
--2021-01-05 10:00:07--  https://cocl.us/Geospatial_data/
Connecting to cocl.us (cocl.us)|169.63.96.176|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-01-05 10:00:08--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.29.197
Connecting to ibm.box.com (ibm.box.com)|107.152.29.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-01-05 10:00:08--  https://ibm.box.com/public/static/9afzr

In [21]:
df_cord = pd.read_csv('GeoCord.csv')
df_cord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
df_total=pd.merge(df_join, df_cord, on='Postal Code')
df_total

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
